## Banking Assistant Chatbot

## This notebook creates and runs a chatbot focused on banking services using a neural network model.

In [1]:
# %pip install tensorflow keras nltk flask

## 1. Training the Model

First, let's prepare the data and train a model to classify user questions about banking services.

In [2]:
import nltk
nltk.download('popular')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

# Initialize lists
words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',', ':', ';']

# Load the dataset
with open('banking_data.json', 'r') as f:
    dataset = json.load(f)

# Process each item in the dataset
for intent in dataset["intents"]:
    for pattern in intent["patterns"]:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

# Lemmatize and lowercase each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words")

# Save vocabulary and classes
pickle.dump(words, open('banking_words.pkl', 'wb'))
pickle.dump(classes, open('banking_classes.pkl', 'wb'))

# Create training data
training = []
output_empty = [0] * len(classes)

# Create bag of words for each document
for doc in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])


# Shuffle and convert to numpy array
random.shuffle(training)
training = np.array(training, dtype=object)

# Split into features and labels
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]

print("Training data created")
print("Length of training:", len(training))

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\nisha\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\nisha\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\nisha\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\nisha\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\nisha\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

284 documents
39 classes ['account_balance', 'account_types', 'atm_locations', 'auto_loan', 'bill_pay', 'branch_locations', 'business_banking', 'check_order', 'credit_card', 'credit_score', 'currency_exchange', 'debit_card', 'digital_wallet', 'direct_deposit', 'dispute_transaction', 'fees', 'financial_planning', 'fraud_security', 'goodbye', 'greeting', 'home_equity', 'insurance_products', 'interest_rates', 'international_banking', 'investment_options', 'loan_info', 'lost_card', 'mobile_banking', 'mobile_deposit', 'mortgage', 'online_banking', 'open_account', 'overdraft_protection', 'retirement_planning', 'statement', 'student_banking', 'tax_information', 'thanks', 'transfer_money']
285 unique lemmatized words
Training data created
Length of training: 284


In [3]:
# Create and train the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

# Save the model
model.save('banking_chatbot_model.h5')
print("Model created and saved")

Epoch 1/200


C:\Users\nisha\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0323 - loss: 3.6799 
Epoch 2/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0328 - loss: 3.6191   
Epoch 3/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0588 - loss: 3.6018   
Epoch 4/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0920 - loss: 3.5438
Epoch 5/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0934 - loss: 3.4525   
Epoch 6/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1822 - loss: 3.2376
Epoch 7/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2012 - loss: 3.1011
Epoch 8/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2895 - loss: 2.8688   
Epoch 9/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2447 - loss: 2.7226
Epoch 10/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3405 - loss: 2.4171
Epoch 11/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4090 - loss: 2.1333
Epoch 12/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy

Model created and saved


## 2. Running the Chatbot

Now let's create the Flask application to serve the banking chatbot.

In [4]:
from keras.models import load_model
from flask import Flask, render_template, request
import threading
import json
import pickle
import random
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np


# Load the trained model and data
model = load_model('banking_chatbot_model.h5')
words = pickle.load(open('banking_words.pkl', 'rb'))
classes = pickle.load(open('banking_classes.pkl', 'rb'))
with open('banking_data.json', 'r') as f:
    dataset = json.load(f)

lemmatizer = WordNetLemmatizer()

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    p = bow(sentence, words)
    res = model.predict(np.array([p]), verbose=0)[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints, intents_json):
    tag = ints[0]['intent'] if ints else None
    if tag:
        for i in intents_json['intents']:
            if i['tag'] == tag:
                return np.random.choice(i['responses'])
    return "I'm sorry, I don't understand. Could you please rephrase your question about banking?"

def chatbot_response(msg):
    ints = predict_class(msg)
    res = get_response(ints, dataset)
    return res

# Test the chatbot
# print("Banking Assistant Chatbot is ready to help you!")
# print("Type 'quit' to exit")
# while True:
#     message = input("You: ")
#     if message.lower() == 'quit':
#         break
#     response = chatbot_response(message)
#     print("Bot:", response)
# def predict_class(sentence):
#     p = bow(sentence, words)
#     res = model.predict(np.array([p]), verbose=0)[0]
#     ERROR_THRESHOLD = 0.25
#     results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
#     results.sort(key=lambda x: x[1], reverse=True)
#     return_list = []
#     for r in results:
#         return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
#     return return_list
# def get_response(ints, intents_json):
#     tag = ints[0]['intent'] if ints else None
#     if tag:
#         for i in intents_json['intents']:
#             if i['tag'] == tag:
#                 return random.choice(i['responses'])
#     return "Sorry, I do not understand."
# def chatbot_response(msg):
#     ints = predict_class(msg)
#     res = get_response(ints, dataset)
#     return res

# Create Flask app
app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")  # NOT "templates/index.html"

@app.route("/get")
def get_bot_response():
    user_text = request.args.get('msg')
    return chatbot_response(user_text)

# To run Flask in a notebook:
def run_flask():
    app.run(debug=False, use_reloader=False)

# Start the server in a background thread (run this cell ONCE)
threading.Thread(target=run_flask).start()

In [5]:
app = Flask(__name__)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/get')
def get_bot_response():
    user_text = request.args.get('msg')
    return chatbot_response(user_text)

# This will run the Flask app when executed directly
if __name__ == '__main__':
    # Start the Flask app in a separate thread
    threading.Thread(target=lambda: app.run(debug=False, use_reloader=False)).start()
    print("Web interface is running at http://127.0.0.1:5000/")


Web interface is running at http://127.0.0.1:5000/
 * Serving Flask app '__main__'
 * Serving Flask app '__main__'
 * Debug mode: off
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Sep/2025 15:02:31] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Sep/2025 15:02:31] "GET /static/styles/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [07/Sep/2025 15:02:32] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Sep/2025 15:02:41] "GET /get?msg=Hello HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Sep/2025 15:02:55] "GET /get?msg=Open%20Account HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Sep/2025 15:03:07] "GET /get?msg=Blance%20 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Sep/2025 15:03:20] "GET /get?m